In [4]:
!pip install PySastrawi
!pip install sklearn-pycrfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import re
import pandas as pd
import gdown
import nltk
import numpy as np
import math
import gspread

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

from nltk.tokenize import word_tokenize 
nltk.download('punkt')
nltk.download('stopwords')

#tfidf lib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [9]:
def read_data(url, sheet='sheet1'):
  gs = gc.open_by_url(url).worksheet(sheet).get_all_values()
  gs = pd.DataFrame.from_records(gs)
  gs.columns = gs.iloc[0]
  gs = gs[1:]
  return gs

def write_data(url, result, sheet):
  write = gc.open_by_url(url)
  write = write.worksheet(sheet)
  write.clear()
  set_with_dataframe(worksheet=write, dataframe=result, include_index=False,include_column_header=True, resize=True)
  return

# PreProcessing
def preprocess(text):
    # remove single character 
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    #remove multiple whitespace into single whitespace
    text = re.sub('\s+',' ',text)
    # remove punctuation
    text = re.sub('[^a-zA-Z]', ' ', text)
    # convert to lower case
    text = text.lower()
    # remove special char and digit
    text = re.sub("(\\d|\\W)+"," ",text)
    # remove Whitespace leading & trailing
    text = text.strip()

    # kombinasi nltk dan sastrawi. <untuk ditambahkan list Tala
    stop_factory = StopWordRemoverFactory().get_stop_words() #load defaul stopword
    stopword_add_list1 = stopword_list_tala #menambahkan stopword
    data = stop_factory + stopword_add_list1

    dictionary = ArrayDictionary(data)
    str = StopWordRemover(dictionary)
    text = str.remove(text)

    # NLTK word tokenize 
    #text = word_tokenize(text)

    return text

def token(text):
  text = word_tokenize(text)
  return text
    #text = word_tokenize(text)

# tfidf function
#Function for sorting tf_idf in descending order
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
        
    return results

# NLTK word tokenize 
def token(text):
  text = word_tokenize(text)
  return text
    #text = word_tokenize(text)
    
def get_top_phrase(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,3),  
            max_features=2000).fit([corpus])
    bag_of_words = vec1.transform([corpus])
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]

## Loading Dataset & preprocessing

In [10]:
url_pl = "https://docs.google.com/spreadsheets/d/19VIlyxleK14LjZCPls-4F9fjg0Sh46Q8oTk5UawJzhI/edit#gid=622809212"
pl = read_data(url_pl,'PL')
pl = pl[['Sub System No','Discipline','Category','Punchlist No','Punchlist Description','Status','Phase','Raised By']] 

In [13]:
pd = pl['Punchlist Description']

In [12]:
pl.head(3)

,Sub System No,Discipline,Category,Punchlist No,Punchlist Description,Status,Phase,Raised By
1,,M,C,PL-C-223-58202,Perform step-5 till finish during start up (af...,Outstanding,COM,Yudi CSU
2,,M,C,PL-C-223-58203,Prove remaining acceptance criteria during sta...,Outstanding,COM,Yudi CSU
3,,M,C,PL-C-470-58204,To test flowing HC from KO drum to CWS after g...,Outstanding,COM,Yudi CSU


# TF-IDF

In [ ]:
cv=CountVectorizer(max_df=0.8, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(pd)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)

# extract from random title
feature_names=cv.get_feature_names()

In [19]:
tfidf = pd.DataFrame()
i=0
for i in pl.index:
  doc=pd[i]
  tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
  sorted_items=sort_coo(tf_idf_vector.tocoo())
  keywords=extract_topn_from_vector(feature_names,sorted_items,10) # format keywprd dictionary. bagaimana menyimpan dictionary dalam dataframe?
  keyword_list = pd.DataFrame.from_dict(keywords,orient='index').index.values.tolist() # merubah dalam format list
  tfidf = tfidf.append(pd.DataFrame([keyword_list]))

tfidf = tfidf.reset_index(drop=True)
tfidf = tfidf.rename(columns={0:"p1",1:"p2",2:"p3",3:"p4",4:"p5",5:"p6",6:"p7",7:"p8",8:"p9",9:"p10"})

# Menggabungkan dalam 1 dataframe
predict_tfidf = pd.concat([df, tfidf], axis=1)
predict_tfidf = predict_tfidf.drop(columns=["tanggal","judul","isi","body_clean","body_token"])
#predict_tfidf.head()

#write_data(url_data, predict_tfidf, 'tfidf_pred')

AttributeError: ignored

#TEXTRANK

In [ ]:
def building_graph(vocab_len, processed_text, vocabulary):
  weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)
  score = np.zeros((vocab_len),dtype=np.float32)
  window_size = 3  
  covered_coocurrences = []

  for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])

  inout = np.zeros((vocab_len),dtype=np.float32)
  for i in range(0,vocab_len):
     for j in range(0,vocab_len):
       inout[i]+=weighted_edge[i][j]
  
   #Scoring Vertices
  MAX_ITERATIONS = 50
  d=0.85
  threshold = 0.0001 #convergence threshold
   
  for iter in range(0,MAX_ITERATIONS):
     prev_score = np.copy(score)
    
     for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)

 
     if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print("Converging at iteration "+str(iter)+"....")
        break

  return vocabulary, score

In [ ]:
def scoring_frase(unique_phrases, vocabulary):
    phrase_scores = []
    keywords = []
    for phrase in unique_phrases:
        phrase_score=0
        keyword = ''
        for word in phrase:
            keyword += str(word)
            keyword += " "
            phrase_score+=score[vocabulary.index(word)] # fungsi score ada di dalam fungsi building_graph, bentuknya ndarray 30, posisi Bug
        phrase_scores.append(phrase_score)
        keywords.append(keyword.strip())

    return keywords, phrase_scores

In [ ]:
# prediksi untuk keyword unigram
predict_unigram = pd.DataFrame()
predict_phrase = pd.DataFrame()
predict_textrank = pd.DataFrame()

i=0
m = 10 # top keyword/unigram/bigram/trigram yg akan diolah

for i in df.index:
#for i in range(10):
  # 1. Unigram Detection
  processed_text= df["body_token"][i]
  vocabulary = list(set(processed_text))
  vocab_len = len(vocabulary)

  # 1a. menghitung basis scoring unigram kata 10 terbesar
  vocabulary, score = building_graph(vocab_len, processed_text, vocabulary) # fungsi score ada masalah dimensi ndarray

  # 1b. membuat dataframe utk membuat top m/10 terbesar. bentuk portrait (turun ke bawah)
  unigram = pd.concat([pd.DataFrame(vocabulary),pd.DataFrame(score)], axis=1, ignore_index=True)
  unigram = unigram[:m]

  # 1c. menyesuaikan dataframe dengan format df dataset landscape (memanjang, kekanan)
  unigram.values.tolist()
  x1 = unigram[0].values.tolist()
  predict_unigram = predict_unigram.append(pd.DataFrame([x1]))

  # 2. Bigram and trigram detection
  list_bigram_trigram = get_top_phrase(df["body_clean"][i], n=50) #Frase tidak menghitung stopword, dalam bentuk yg belum tertokenise), agar urutan kata masih bisa di olah
  phrases = pd.DataFrame(list_bigram_trigram)
  phrases = phrases[phrases[1]>2] # minimum phrase muncul berapa kali 3x keatas, subject utk di tuning

  #2a2. Ditokenisasi untuk memudahkan penghitungan score dari kata dasarnya
  phrases[1] = phrases[0].apply(token) #membuat dataframe tambahan untuk frase, utk di tokenisasi
  unique_phrases = phrases[1].values.tolist() # konvert frase ke dalam list

  #2a1. panggil fungsi penghitung keyword dan phrase_score
  keywords, phrase_scores = scoring_frase(unique_phrases, vocabulary)

  #2b1. membuat dataframe utk membuat top n/10 terbesar. bentuk portrait (turun ke bawah)
  tbl_score_frase = pd.concat([pd.DataFrame(keywords),pd.DataFrame(phrase_scores)], axis=1, ignore_index=True)

  #2b2. membuat percabangan saat dimensi dataframe (0,0) maka tetap di rename
  if tbl_score_frase.empty:
    tbl_score_frase = pd.DataFrame([0,0])

  #tbl_score_frase.columns = ['keyword','score'] #### BUG 02 perlu di buat fungsi jika tidak ketemu frase maka, perlu teknik biar tidak perlu rename sebanyak iterasi i 
  tbl_score_frase = tbl_score_frase[:m]

  #2c. menyesuaikan dataframe dengan format df dataset landscape (memanjang, kekanan)
  tbl_score_frase.values.tolist()
  x2 = tbl_score_frase[0].values.tolist()
  predict_phrase = predict_phrase.append(pd.DataFrame([x2]))

  #3. Merge unigram/bigram/trigram
  merge = pd.concat([unigram, tbl_score_frase]).sort_values(by=[1], ascending = False).reset_index(drop=True)
  merge = merge[:m]
  merge = merge[0].values.tolist() 

  predict_textrank = predict_textrank.append(pd.DataFrame([merge]))
  #predict_textrank

Converging at iteration 28....
Converging at iteration 28....
Converging at iteration 30....
Converging at iteration 27....
Converging at iteration 25....
Converging at iteration 19....
Converging at iteration 29....
Converging at iteration 28....
Converging at iteration 26....
Converging at iteration 28....
Converging at iteration 29....
Converging at iteration 29....
Converging at iteration 31....
Converging at iteration 29....
Converging at iteration 24....
Converging at iteration 25....
Converging at iteration 27....
Converging at iteration 30....
Converging at iteration 29....
Converging at iteration 22....
Converging at iteration 21....
Converging at iteration 20....
Converging at iteration 23....
Converging at iteration 31....
Converging at iteration 29....
Converging at iteration 27....
Converging at iteration 22....
Converging at iteration 25....
Converging at iteration 15....
Converging at iteration 21....
Converging at iteration 28....
Converging at iteration 22....
Convergi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Converging at iteration 24....
Converging at iteration 27....
Converging at iteration 25....
Converging at iteration 33....
Converging at iteration 23....
Converging at iteration 23....
Converging at iteration 26....
Converging at iteration 29....
Converging at iteration 22....
Converging at iteration 27....
Converging at iteration 31....
Converging at iteration 28....
Converging at iteration 27....
Converging at iteration 27....
Converging at iteration 20....
Converging at iteration 25....
Converging at iteration 22....
Converging at iteration 29....
Converging at iteration 27....
Converging at iteration 20....
Converging at iteration 27....
Converging at iteration 32....
Converging at iteration 27....
Converging at iteration 24....
Converging at iteration 28....
Converging at iteration 24....
Converging at iteration 27....
Converging at iteration 29....
Converging at iteration 27....
Converging at iteration 26....
Converging at iteration 26....
Converging at iteration 22....
Convergi

In [ ]:
textrank = predict_textrank.reset_index(drop=True)
textrank = textrank.rename(columns={0:"p1",1:"p2",2:"p3",3:"p4",4:"p5",5:"p6",6:"p7",7:"p8",8:"p9",9:"p10"})

In [ ]:
# Menggabungkan dalam 1 dataframe
textrank = pd.concat([df, textrank], axis=1)
textrank = textrank.drop(columns=["tanggal","judul","isi","body_clean","body_token"])

In [ ]:
write_data(url_data, textrank, 'textrank_pred')